In [1]:
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot as plt
%matplotlib

from tqdm import tqdm

Using matplotlib backend: TkAgg


# Build a recommender using PCA tools

Mostly to see how they work

In [2]:
engine = sql.create_engine('sqlite:///instacart.db')
db = {}

In [3]:
for table in ['order_products', 'departments']:
    db[table] = pd.read_sql_table(table, engine, index_col=0,)

In [6]:
orders = db['order_products'].groupby('order_id')['product_id'].unique()   # orders = list of products in each order

In [23]:
pca_frame = pd.read_csv('product_pca_coords.csv', index_col=0)
pca_frame.values[:, :20] = pca_frame.values[:, :20].astype('float')

In [96]:
def vector_best_distance(product_id, output='scores'):
    try:
        print('best match for:', pca_frame.product_name.loc[product_id])
        my_coords = pca_frame.loc[product_id].values[:20].astype('float32')
        actual_index = pca_frame.index.get_loc(product_id)
        # print(actual_index)
    except KeyError:
        print('invalid item')
        return None
    
    diff = pca_frame.values[:, :20].astype('float32') - my_coords
    # print(diff.shape, diff.dtype)
    scores = pd.np.linalg.norm(diff, axis=1)
    scores[actual_index] = pd.np.Inf
    # print(scores[:5])

    best = pd.np.argmin(scores)
    if output == 'scores':
        print(pca_frame.product_name.iloc[best])
        return scores
    else:
        return best, pca_frame.product_name.iloc[best], pca_frame.index[best], pd.np.min(scores)

In [101]:
def vector_distance(product_id, target_id):
    try:
        print('distance from:', pca_frame.product_name.loc[product_id], ' to ', pca_frame.product_name.loc[target_id])
        my_coords = pca_frame.loc[product_id].values[:20].astype('float32')
        target = pca_frame.loc[target_id].values[:20].astype('float32')
        # print(actual_index)
    except KeyError:
        print('invalid item')
        return None
    
    return pd.np.linalg.norm(my_coords - target)

In [102]:
vector_best_distance(1)

best match for: Chocolate Sandwich Cookies
Wheat Thins Original


array([     inf, 9.402093, 9.277661, ..., 9.189031, 9.491742, 8.685622],
      dtype=float32)

In [105]:
vector_distance(1, 5)

distance from: Chocolate Sandwich Cookies  to  Green Chile Anytime Sauce


9.518086

In [96]:
def vector_best_distance(product_id, output='scores'):
    try:
        print('best match for:', pca_frame.product_name.loc[product_id])
        my_coords = pca_frame.loc[product_id].values[:20].astype('float32')
        actual_index = pca_frame.index.get_loc(product_id)
        # print(actual_index)
    except KeyError:
        print('invalid item')
        return None
    
    diff = pca_frame.values[:, :20].astype('float32') - my_coords
    # print(diff.shape, diff.dtype)
    scores = pd.np.linalg.norm(diff, axis=1)
    scores[actual_index] = pd.np.Inf
    # print(scores[:5])

    best = pd.np.argmin(scores)
    if output == 'scores':
        print(pca_frame.product_name.iloc[best])
        return scores
    else:
        return best, pca_frame.product_name.iloc[best], pca_frame.index[best], pd.np.min(scores)

In [91]:
for i in range(20):
    print(vector_best_distance(i, output=None))
    print()

invalid item
None

best match for: Chocolate Sandwich Cookies
(8870, 'Wheat Thins Original', 10310, 1.5335858)

best match for: All-Seasons Salt
(24809, 'Uncured Bacon & Eggs Scramble Bowl Frozen Meal', 28755, 0.9662164)

best match for: Robust Golden Unsweetened Oolong Tea
(24273, 'Yam Good Paleo Muffins', 28129, 1.1087532)

best match for: Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce
(23356, 'Traditional Crust Pepperoni Pizza', 27062, 1.4652718)

best match for: Green Chile Anytime Sauce
(40745, 'MegaFlora Capsules', 47302, 0.35953498)

invalid item
None

best match for: Pure Coconut Water With Orange
(33544, 'FiberWell Fit Adult Gummies', 38917, 0.74024016)

best match for: Cut Russet Potatoes Steam N' Mash
(15909, 'Thin Spaghetti', 18423, 1.0751762)

best match for: Light Strawberry Blueberry Yogurt
(36654, 'Dishwand', 42529, 1.0679736)

best match for: Sparkling Orange Juice & Prickly Pear Beverage
(19311, 'Total 0% Cherry Nonfat Yogurt', 22367, 2.7002997)

be

In [112]:
def generate_recs(i):
    try:
        basket = orders.iloc[i]
        last = basket[-1]
        basket = basket[:-1]
    except KeyError:
        print('order_id {} may exist in orders.index, but contains no items. \n'.format(orders.index[i]))
        return None
    
    print('\n' + '+ ' * 30 + '\n')
    print(pca_frame.loc[basket].product_name)

    score = 0
    for item in basket:
        score += vector_best_distance(item)
    
    best = pd.np.argmin(score)
    product = pca_frame.product_name.iloc[best]
    print()
    print('we chose:')
    print(product)

    print()
    print('actual final item: {}'.format(pca_frame.loc[last].product_name))
    
    print()
    print('distance between items:', vector_distance(pca_frame.index[best], last))
        
    print()
    return item

In [114]:
for i in range(20):
    generate_recs(i)


+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

product_id
33120                          Organic Egg Whites
28985                       Michigan Organic Kale
9327                                Garlic Powder
45918                              Coconut Butter
30035                           Natural Sweetener
17794                                     Carrots
40141            Original Unflavored Gelatine Mix
1819     All Natural No Stir Creamy Almond Butter
Name: product_name, dtype: object
best match for: Organic Egg Whites
Organic Strawberry Fruit Spread
best match for: Michigan Organic Kale
Organic Small Bunch Celery
best match for: Garlic Powder
Romaine Lettuce
best match for: Coconut Butter
Chocolate Coconut Milk Beverage
best match for: Natural Sweetener
The People's Pickle Garlic Dill Pickle Slices
best match for: Carrots
Organic Whole Milk
best match for: Original Unflavored Gelatine Mix
Anaheim Pepper
best match for: All Natural No Stir Creamy Almond Butter
Maple A

Large Lemon
best match for: Organic Avocado
Asparagus
best match for: No Salt Added Black Beans
100% Recycled Paper Towels

we chose:
Organic Red Radish, Bunch

actual final item: Jalapeno Peppers

distance from: Organic Red Radish, Bunch  to  Jalapeno Peppers
distance between items: 4.8321915


+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

product_id
31974    Cinnamon Sugar Cookie Chips
14159            Seedless Watermelon
33357          Fresh Sugar Snap Peas
38996                 Grape Tomatoes
Name: product_name, dtype: object
best match for: Cinnamon Sugar Cookie Chips
Diet The Original Cream Soda
best match for: Seedless Watermelon
Shredded Mozzarella Cheese
best match for: Fresh Sugar Snap Peas
Turkey Store Lean Ground Turkey
best match for: Grape Tomatoes
No Pulp Calcium & Vitamin D Pure Premium 100% Pure Orange Juice

we chose:
Trop50 No Pulp Orange Juice w/ Calcium + Vitamin D, 50% Less Sugar & Calories

actual final item: Mini Sweet Peppers

distance from: Tro

Goat's Milk Cheese
best match for: Mini Seedless Cucumbers
Extra Sharp White Cheddar
best match for: Total 0% Nonfat Plain Greek Yogurt
Total 2% All Natural Plain Greek Yogurt
best match for: Vanilla Milk Chocolate Almond Ice Cream Bars
Mint Chocolate Chip Ice Cream
best match for: Roasted Sesame Seaweed Snacks
Roasted Chicken Breast
best match for: Fruit Roll-Ups Strawberry Sensation Fruit Flavored Snacks
Homemade Style Pizza Sauce
best match for: All Natural Virgin Lemonade
Mild Cheddar Cheese
best match for: Original Potato Chips
Chicken Flavor Ramen Noodle Soup

we chose:
Grapefruit Juice

actual final item: Vanilla Milk Chocolate Ice Cream Bars

distance from: Grapefruit Juice  to  Vanilla Milk Chocolate Ice Cream Bars
distance between items: 12.966664


+ + + + + + + + + + + + + + + + + + + + + + + + + + + + + + 

product_id
14678      Organic Frozen Peas
19678    Organic Russet Potato
22935     Organic Yellow Onion
21903     Organic Baby Spinach
33731          Grated Parmesan
Na

# Recommender works... ok!

Recommendations are generally reasonable and fall in the space of products that these consumers may purchse.

### Things to improve:

This system only uses our PCA distance metric. It could definitely be improved by using other available data:
1. Department/Aisle info
2. Product add-to-order sequence data
3. Time of day/week information
4. (customer purchase history if this was for real deployment)
5. Text from product description